<a href="https://colab.research.google.com/github/beastboy-93/wildecho/blob/main/Copy_of_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required libraries
!pip install librosa tensorflow

# Import necessary libraries
import numpy as np
import os
import librosa
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Path and parameters
DATASET_PATH = "/content/drive/MyDrive/sounds"
N_MELS, MAX_PAD_LEN, SAMPLE_RATE = 128, 174, 22050

# Mel spectrogram extraction
def extract_mel_spectrogram(file_path):
    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE)
        y, _ = librosa.effects.trim(y, top_db=20)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS, n_fft=2048, hop_length=256, fmax=8000)
        mel_spec_db = librosa.power_to_db(mel_spec + 1e-9, ref=np.max)
        mel_spec_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db) + 1e-9)
        mel_spec_padded = np.zeros((N_MELS, MAX_PAD_LEN))
        mel_spec_padded[:, :min(MAX_PAD_LEN, mel_spec_norm.shape[1])] = mel_spec_norm[:, :MAX_PAD_LEN]
        return mel_spec_padded
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None  # Return None for invalid files

# Data processing
X, y, labels = [], [], {}
for idx, animal in enumerate(os.listdir(DATASET_PATH)):
    animal_folder = os.path.join(DATASET_PATH, animal)
    if os.path.isdir(animal_folder):
        labels[idx] = animal
        for file in filter(lambda f: f.endswith(".wav"), os.listdir(animal_folder)):
            file_path = os.path.join(animal_folder, file)
            feature = extract_mel_spectrogram(file_path)
            if feature is not None:
                X.append(feature)
                y.append(idx)
            else:
                print(f"Skipping file due to processing error: {file_path}")

print(labels)

# Convert to numpy arrays
if X:
    X, y = np.array(X)[..., np.newaxis], to_categorical(y, num_classes=len(labels))
else:
    raise ValueError("No valid spectrograms found in dataset.")

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=np.argmax(y, axis=1))
class_weight_dict = dict(enumerate(compute_class_weight('balanced', classes=np.unique(np.argmax(y, axis=1)), y=np.argmax(y, axis=1))))

# CNN Model
def create_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'), BatchNormalization(), Dropout(0.5),
        Dense(256, activation='relu'), Dropout(0.3), Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model((N_MELS, MAX_PAD_LEN, 1), len(labels))
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test),
          class_weight=class_weight_dict,
          callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1),
                     EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, verbose=1),
                     ModelCheckpoint('/content/drive/MyDrive/best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)],
          verbose=1)

# Predictions
def predict_animal(AUDIO_FILE):
    feature = extract_mel_spectrogram(AUDIO_FILE)
    if feature is not None:
        spectrogram = feature[np.newaxis, ..., np.newaxis]
        predictions = model.predict(spectrogram)
        predicted_label = np.argmax(predictions)
        print(f"Predicted Animal: {labels[predicted_label]} (Confidence: {predictions[0][predicted_label]:.2f})")
    else:
        print(f"Error: Could not process {AUDIO_FILE}")

for AUDIO_FILE in ["/content/drive/MyDrive/test_as/Lion_8.wav", "/content/drive/MyDrive/test_as/Monkey_9.wav",
                   "/content/drive/MyDrive/test_as/Sheep_11.wav", "/content/drive/MyDrive/test_as/Cow_8.wav",
                   "/content/drive/MyDrive/test_as/Horse_10.wav"]:
    predict_animal(AUDIO_FILE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{0: 'Elephant', 1: 'Bear', 2: 'Horse', 3: 'Lion', 4: 'Dog', 5: 'Donkey', 6: 'Cat', 7: 'Sheep', 8: 'Cow', 9: 'Monkey'}
Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.2049 - loss: 3.0866
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to /content/drive/MyDrive/best_model.h5


57/57 ━━━━━━━━━━━━━━━━━━━━ 17s 117ms/step - accuracy: 0.2064 - loss: 3.0773 - val_accuracy: 0.1000 - val_loss: 2.5502 - learning_rate: 1.0000e-04
Epoch 2/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5811 - loss: 1.3800
Epoch 2: val_accuracy did not improve from 0.10000
57/57 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.5812 - loss: 1.3796 - val_accuracy: 0.1000 - val_loss: 3.1508 - learning_rate: 1.0000e-04
Epoch 3/50
53/57 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6867 - loss: 0.9741
Epoch 3: val_accuracy did not improve from 0.10000
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6860 - loss: 0.9781 - val_accuracy: 0.1000 - val_loss: 3.8055 - learning_rate: 1.0000e-04
Epoch 4/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7524 - loss: 0.7775
Epoch 4: val_accuracy did not improve from 0.10000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7524 - loss: 0.7794 - val_accuracy: 0.1000 - val_loss: 3.9791 - learning_rate: 1.0000e-04
Epoch 5/50
56/

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.7676 - loss: 0.6549 - val_accuracy: 0.1800 - val_loss: 4.4410 - learning_rate: 1.0000e-04
Epoch 6/50
54/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8621 - loss: 0.4769
Epoch 6: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 6: val_accuracy did not improve from 0.18000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8608 - loss: 0.4771 - val_accuracy: 0.1600 - val_loss: 4.2011 - learning_rate: 1.0000e-04
Epoch 7/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8888 - loss: 0.3755
Epoch 7: val_accuracy improved from 0.18000 to 0.38000, saving model to /content/drive/MyDrive/best_model.h5


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.8888 - loss: 0.3755 - val_accuracy: 0.3800 - val_loss: 2.9533 - learning_rate: 5.0000e-05
Epoch 8/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9014 - loss: 0.3163
Epoch 8: val_accuracy did not improve from 0.38000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9015 - loss: 0.3162 - val_accuracy: 0.3800 - val_loss: 2.2042 - learning_rate: 5.0000e-05
Epoch 9/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9520 - loss: 0.2299
Epoch 9: val_accuracy improved from 0.38000 to 0.48000, saving model to /content/drive/MyDrive/best_model.h5


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9520 - loss: 0.2299 - val_accuracy: 0.4800 - val_loss: 1.5266 - learning_rate: 5.0000e-05
Epoch 10/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9567 - loss: 0.1960
Epoch 10: val_accuracy improved from 0.48000 to 0.60000, saving model to /content/drive/MyDrive/best_model.h5


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.9777 - loss: 0.1155 - val_accuracy: 0.8200 - val_loss: 0.5692 - learning_rate: 5.0000e-05
Epoch 13/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9757 - loss: 0.1325
Epoch 13: val_accuracy improved from 0.82000 to 0.84000, saving model to /content/drive/MyDrive/best_model.h5


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.9758 - loss: 0.1323 - val_accuracy: 0.8400 - val_loss: 0.4636 - learning_rate: 5.0000e-05
Epoch 14/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9864 - loss: 0.1056
Epoch 14: val_accuracy improved from 0.84000 to 0.86000, saving model to /content/drive/MyDrive/best_model.h5


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9862 - loss: 0.1059 - val_accuracy: 0.8600 - val_loss: 0.4849 - learning_rate: 5.0000e-05
Epoch 15/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9949 - loss: 0.0732
Epoch 15: val_accuracy did not improve from 0.86000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9946 - loss: 0.0739 - val_accuracy: 0.8600 - val_loss: 0.4419 - learning_rate: 5.0000e-05
Epoch 16/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9902 - loss: 0.0741
Epoch 16: val_accuracy did not improve from 0.86000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9904 - loss: 0.0742 - val_accuracy: 0.8600 - val_loss: 0.4330 - learning_rate: 5.0000e-05
Epoch 17/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9944 - loss: 0.0643
Epoch 17: val_accuracy did not improve from 0.86000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9942 - loss: 0.0644 - val_accuracy: 0.8200 - val_loss: 0.4499 - learning_rate: 5.0000e-05
Epoch 18/5

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9992 - loss: 0.0437 - val_accuracy: 0.9000 - val_loss: 0.3882 - learning_rate: 5.0000e-05
Epoch 23/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9951 - loss: 0.0383
Epoch 23: val_accuracy did not improve from 0.90000
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9950 - loss: 0.0385 - val_accuracy: 0.8400 - val_loss: 0.3839 - learning_rate: 5.0000e-05
Epoch 24/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9897 - loss: 0.0429
Epoch 24: val_accuracy did not improve from 0.90000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9895 - loss: 0.0434 - val_accuracy: 0.8800 - val_loss: 0.4054 - learning_rate: 5.0000e-05
Epoch 25/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9926 - loss: 0.0521
Epoch 25: val_accuracy did not improve from 0.90000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9924 - loss: 0.0524 - val_accuracy: 0.8600 - val_loss: 0.4218 - learning_rate: 5.0000e-05
Epoch 26/5

57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9945 - loss: 0.0337 - val_accuracy: 0.9200 - val_loss: 0.3959 - learning_rate: 2.5000e-05
Epoch 32/50
55/57 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0229
Epoch 32: val_accuracy did not improve from 0.92000
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 1.0000 - loss: 0.0231 - val_accuracy: 0.9200 - val_loss: 0.4100 - learning_rate: 2.5000e-05
Epoch 33/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9955 - loss: 0.0309
Epoch 33: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.

Epoch 33: val_accuracy did not improve from 0.92000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9956 - loss: 0.0309 - val_accuracy: 0.8600 - val_loss: 0.4223 - learning_rate: 2.5000e-05
Epoch 34/50
56/57 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0253
Epoch 34: val_accuracy did not improve from 0.92000
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 1.0000 - loss: 0.0255 - 

In [23]:
for AUDIO_FILE in ["/content/drive/MyDrive/test_as/Bear_11.wav", "/content/drive/MyDrive/test_as/Donkey_12.wav",
                   "/content/drive/MyDrive/test_as/Elephant_12.wav", "/content/drive/MyDrive/test_as/Cat_51.wav",
                   "/content/drive/MyDrive/test_as/Dog_52.wav"]:
    predict_animal(AUDIO_FILE)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Animal: Bear (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Animal: Donkey (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Animal: Elephant (Confidence: 1.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted Animal: Cat (Confidence: 0.66)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Predicted Animal: Dog (Confidence: 0.93)
